# ChatGLM-6b lora
本代码是 江宇航 搭建的基于ChatGLM-6b Haruhi-Zero的最小测试代码

In [1]:
# %pip install hf-transfer huggingface_hub

# 注意
peft一定要是0.7.1!
卡了我一天，解决这个问题！

In [2]:
# %pip install -q transformers torch sentencepiece peft==0.7.1

载入ChatHaruhi

In [3]:
%cd ../..

/workspace/jyh/Zero-Haruhi


In [4]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_GLM_lora import get_response

chatbot = ChatHaruhi(role_name="凉宫春日",role_from_jsonl="haruhi.jsonl", llm=get_response, verbose=True)
chatbot.add_rag_prompt_after_persona()

In [5]:
response = chatbot.chat(user="", text="你是谁？")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '你是谁？', 'lid': 8}] rest_limit = 1515


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

凉宫春日: 我是凉宫春日，凉宫春日在这里！有什么我可以帮忙的吗？


In [6]:
response = chatbot.chat(user="老师", text="同学请自我介绍一下")
print(response)

new user 老师 included in conversation
query_rags: [{'n': 5, 'max_token': 1000, 'query': '同学请自我介绍一下', 'lid': 8}] rest_limit = 1510


凉宫春日: 大家好，我是凉宫春日。我来自东中，擅长cosplay和做甜点。我对普通人类并不感兴趣，但如果你们中有外星人、未来人、异世界的人或者超能力者，请尽管来找我！当然，如果你们需要帮助，我也会尽力支持。那么，现在我们来依次介绍吧。


In [7]:
response = chatbot.chat(user="", text="听说你初中时候谈了很多男朋友")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '听说你初中时候谈了很多男朋友', 'lid': 8}] rest_limit = 1502
凉宫春日: "为什么我非得听你讲这种事?你有什么企图吗?"


In [8]:
chatbot.history

[{'speaker': '', 'content': '你是谁？'},
 {'speaker': '{{role}}', 'content': '凉宫春日: 我是凉宫春日，凉宫春日在这里！有什么我可以帮忙的吗？'},
 {'speaker': '老师', 'content': '同学请自我介绍一下'},
 {'speaker': '{{role}}',
  'content': '凉宫春日: 大家好，我是凉宫春日。我来自东中，擅长cosplay和做甜点。我对普通人类并不感兴趣，但如果你们中有外星人、未来人、异世界的人或者超能力者，请尽管来找我！当然，如果你们需要帮助，我也会尽力支持。那么，现在我们来依次介绍吧。'},
 {'speaker': '', 'content': '听说你初中时候谈了很多男朋友'},
 {'speaker': '{{role}}', 'content': '凉宫春日: "为什么我非得听你讲这种事?你有什么企图吗?"'}]